In [7]:
import pandas as pd
import numpy as np

In [8]:
csv_file_path = "batdongsan.csv"
normalized_csv_path = "batdongsan_normalized.csv"
df = pd.read_csv(csv_file_path, sep=';', encoding='utf-8-sig')
print(f"Số dòng dữ liệu gốc: {len(df)}")

# xem dữ liệu các giá trị khuyết thiếu của từng cột
print(df.isnull().sum())

Số dòng dữ liệu gốc: 2924
title         193
price           1
area            0
bedroom      1280
toilet       1423
link            0
address        26
legal         407
furniture    1420
post_date      26
loai_bds        0
district     2924
dtype: int64


In [9]:
# Chuẩn hóa 'N/A' thành giá trị thiếu chuẩn (NaN)
df.replace('N/A', np.nan, inplace=True)
df.replace('="N/A"', np.nan, inplace=True)

In [10]:
# CHUYỂN ĐỔI DỮ LIỆU SANG DẠNG SỐ ---
def clean_price(price_str):
    if pd.isnull(price_str) or 'giá thỏa thuận' in str(price_str).lower():
        return np.nan
    price_str = str(price_str).lower().replace(',', '.')
    if 'tỷ' in price_str:
        value = price_str.replace('tỷ', '').strip()
        try: return float(value)
        except ValueError: return np.nan
    elif 'triệu' in price_str:
        value = price_str.replace('triệu', '').strip()
        try: return float(value) / 1000
        except ValueError: return np.nan
    return np.nan

def clean_area(area_str):
    if pd.isnull(area_str): return np.nan
    try: return float(str(area_str).replace('m²', '').strip())
    except ValueError: return np.nan

def clean_rooms(room_str):
    if pd.isnull(room_str): return np.nan # Giữ là NaN để xử lý ở bước sau
    try: return int(''.join(filter(str.isdigit, str(room_str))))
    except ValueError: return np.nan


df['price_ty'] = df['price'].apply(clean_price)
df['area_m2'] = df['area'].apply(clean_area)
df['bedrooms_num'] = df['bedroom'].apply(clean_rooms)
df['toilets_num'] = df['toilet'].apply(clean_rooms)

df.drop(columns=['price', 'area', 'bedroom', 'toilet'], inplace=True)


In [11]:
# --- XỬ LÝ GIÁ TRỊ THIẾU ---

# Điền giá trị thiếu bằng TRUNG VỊ (median)
median_price = df['price_ty'].median()
median_area = df['area_m2'].median()
median_bedrooms = df['bedrooms_num'].median()
median_toilets = df['toilets_num'].median()

df['price_ty'] = df['price_ty'].fillna(median_price)
df['area_m2'] = df['area_m2'].fillna(median_area)
df['bedrooms_num'] = df['bedrooms_num'].fillna(median_bedrooms)
df['toilets_num'] = df['toilets_num'].fillna(median_toilets)

# Các cột dạng chuỗi: điền 'Missing' nếu thiếu
df['address'] = df['address'].fillna('Missing')
df['district'] = df['address'].apply(
    lambda x: x.split(',')[-2].strip() if isinstance(x, str) and len(x.split(',')) > 1 else 'Missing'
)
df['title'] = df['title'].fillna('Missing')
df['loai_bds'] = df['loai_bds'].fillna('Missing')
df['furniture'] = df['furniture'].fillna('Missing')
df['legal'] = df['legal'].fillna('Missing')
df['post_date'] = df['post_date'].fillna('Missing')
df['link'] = df['link'].fillna('Missing')

In [15]:
# Kiểm tra lại xem còn giá trị thiếu không
print("\nKiểm tra lại dữ liệu sau khi chuẩn hóa:")
print(df.isnull().sum())

df.to_csv(normalized_csv_path, index=False, encoding='utf-8-sig')

print(f"Hoàn thành! Dữ liệu đã chuẩn hóa được lưu tại: {normalized_csv_path}")
print(f"Tổng số dữ liệu cuối cùng: {len(df)}")


Kiểm tra lại dữ liệu sau khi chuẩn hóa:
title           0
link            0
address         0
legal           0
furniture       0
post_date       0
loai_bds        0
district        0
price_ty        0
area_m2         0
bedrooms_num    0
toilets_num     0
dtype: int64
Hoàn thành! Dữ liệu đã chuẩn hóa được lưu tại: batdongsan_normalized.csv
Tổng số dữ liệu cuối cùng: 2924
